# 00_import_libraries

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

# 01_data_reading

In [2]:
path = Path.cwd() / '01_data_in' / 'base_maestra.csv'
df_maestra = pd.read_csv(path, sep='\t')
path = Path.cwd() / '01_data_in' / 'base_transaccional.csv'
df_transaccional = pd.read_csv(path, sep='\t')

# 02_transformation

In [ ]:
# 01_limpieza_transaccional
df_transaccional['Fecha_Consolidada'] = df_transaccional['Fecha_Consolidada'].str.lstrip('-')
df_transaccional['Fecha_Consolidada'] = pd.to_datetime(df_transaccional['Fecha_Consolidada'], errors='coerce')

df_transaccional['Codigo_Cliente_Transaccional'] = 'CA-' + df_transaccional['Codigo_Cliente_Transaccional'].str.strip()

df_transaccional['Venta_Cliente'] = (
    df_transaccional['Venta_Cliente']
    .str.replace('$', '', regex=False)
    .str.replace('COP', '', regex=False)
    .str.replace('.', '', regex=False) # Elimina los puntos de miles
    .str.replace(',', '.', regex=False) # Convierte la coma decimal a punto
    .str.strip()
    .astype(float)
)

# sin sucursal, solo codigo_cliente
# 02_limpieza_maestra
df_maestra.sort_values(by='Status_Atencion', ascending=False, inplace=True)
df_maestra_temp = df_maestra.groupby(['Codigo_Cliente']).agg({'Status_Atencion':'first'}).reset_index()
df_maestra = df_maestra.drop(columns='Status_Atencion')# supuesto, si alguna sucursal no tiene atencion, el cliente no tiene atencion
df_maestra = df_maestra.merge(df_maestra_temp, on='Codigo_Cliente', how='left')
df_maestra.drop_duplicates(subset='Codigo_Cliente', inplace=True)

# 03_merge
list_columns_1 = ['Distribuidora', 'Codigo_Cliente_Transaccional', 'Fecha_Consolidada', 'Venta_Cliente']
list_columns_2 = ['Cliente_Unico', 'Codigo_Cliente', 'Status_Atencion']
df_merged = df_transaccional[list_columns_1].merge(df_maestra[list_columns_2], left_on='Codigo_Cliente_Transaccional', right_on='Codigo_Cliente', how='left')

# 04_estado_cliente
df_merged.sort_values(by='Fecha_Consolidada', ascending=True, inplace=True)

df_temp = (
    df_merged[df_merged['Fecha_Consolidada'].notnull()]
    .groupby('Cliente_Unico')
    .agg({'Codigo_Cliente':'last'})  # toma el último código según fecha asc
    .reset_index()
    .rename(columns={'Codigo_Cliente':'Codigo_Cliente_2'})
)
df_merged = df_merged.merge(df_temp, on='Cliente_Unico', how='left')
df_merged['estado_cliente'] = df_merged.apply(
    lambda row: 'actualizado' if row['Codigo_Cliente'] == row['Codigo_Cliente_2'] else 'desactualizado',
    axis=1
)
df_merged = df_merged[[
    'Distribuidora', 'Codigo_Cliente_Transaccional', 'Fecha_Consolidada',
    'Venta_Cliente', 'Cliente_Unico', 'Codigo_Cliente',
    'Status_Atencion', 'Codigo_Cliente_2', 'estado_cliente'
]]


In [ ]:
path = Path.cwd() / '02_data_out' / 'df_final.csv'
df_merged.to_csv(path, sep='\t')